In [1]:
import pandas as pd # 테이블 형식으로 데이터를 처리할 수 있게 하는 라이브러리
import re
from tqdm import tqdm

In [2]:
from selenium import webdriver as wd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup

import time

In [3]:
url = "https://www.gangdong.go.kr/web/koRenew_3/sisul/uiList.do"
path = "C:/chromedriver.exe" #크롬 웹드라이버 경로

In [14]:
class Gangdong:
    def __init__(self, url):
        self.url = url
        self.df = pd.DataFrame()
        self.driver = None
        self.max_range = '0'

    def setup(self, path):
        options = Options()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
        self.driver = wd.Chrome(executable_path=path, options=options) # 웹드라이버 경로를 executable_path에 설정
        self.driver.get(self.url)
        
    def find_max_page(self):
        text_ = self.driver.find_element_by_xpath('//*[@id="content_focus"]/div[2]/span[2]').text
        self.max_range = re.findall(r"[0-9]+", text_)[0]
        
        
    def read_table(self, i):
        self.driver.find_element_by_xpath(f'//*[@id="content_focus"]/div[4]/a[{i+3}]').click()
        soup = BeautifulSoup(self.driver.page_source)
        table = pd.read_html(str(soup))[0]
        
        self.df = pd.concat([self.df, table])
        
    def load(self):
        
        Gangdong.find_max_page(self)
        
        for x in range(int(self.max_range) // 10 + 1):
            for i in tqdm(range(10)):
                try:
                    Gangdong.read_table(self, i)

                    if i+3 == 12:
                        self.driver.find_element_by_xpath('//*[@id="content_focus"]/div[4]/a[13]').click()
                        
                except:
                    break
        
                time.sleep(2)
                
        self.driver.quit()
        
        return self.df.reset_index(drop=True)

In [15]:
gd = Gangdong(url)
gd.setup(path)
table = gd.load()

 70%|██████████████████████████████████████████████████████████                         | 7/10 [00:18<00:07,  2.64s/it]


In [17]:
table.tail()

,번호,시설명,분류,전화,홈페이지,동,위치
687,1,둔촌현대A 3차경로당,어르신,02-477-6668,NaN,둔촌2동,"서울특별시 강동구 진황도로 211 (둔촌동, 현대3차아파트)지도보기"
688,4,신성둔촌미소지움1차A 경로당,어르신,02-477-9627,NaN,둔촌2동,"서울특별시 강동구 명일로 102 (둔촌동, 신성둔촌미소지움)지도보기"
689,3,신성둔촌미소지움2차A경로당,어르신,02-484-8727,NaN,둔촌2동,"서울특별시 강동구 진황도로 212 (둔촌동, 신성둔촌미소지움2차)지도보기"
690,2,명일우성 A 경로당,어르신,02-481-5748,NaN,명일2동,"서울특별시 강동구 고덕로62길 76 (명일동, 명일우성아파트)지도보기"
691,1,둔촌현대A 3차경로당,어르신,02-477-6668,NaN,둔촌2동,"서울특별시 강동구 진황도로 211 (둔촌동, 현대3차아파트)지도보기"


In [19]:
# 위치 데이터 뒤에 지도보기 글자 제거
table['위치'] = table['위치'].str.rstrip('지도보기')

In [20]:
table.tail()

,번호,시설명,분류,전화,홈페이지,동,위치
687,1,둔촌현대A 3차경로당,어르신,02-477-6668,NaN,둔촌2동,"서울특별시 강동구 진황도로 211 (둔촌동, 현대3차아파트)"
688,4,신성둔촌미소지움1차A 경로당,어르신,02-477-9627,NaN,둔촌2동,"서울특별시 강동구 명일로 102 (둔촌동, 신성둔촌미소지움)"
689,3,신성둔촌미소지움2차A경로당,어르신,02-484-8727,NaN,둔촌2동,"서울특별시 강동구 진황도로 212 (둔촌동, 신성둔촌미소지움2차)"
690,2,명일우성 A 경로당,어르신,02-481-5748,NaN,명일2동,"서울특별시 강동구 고덕로62길 76 (명일동, 명일우성아파트)"
691,1,둔촌현대A 3차경로당,어르신,02-477-6668,NaN,둔촌2동,"서울특별시 강동구 진황도로 211 (둔촌동, 현대3차아파트)"


In [22]:
table.to_excel('./복지시설.xlsx', index=False)

In [ ]:
# User-agent:*
# Disallow:/ap=B0007$
# Disallow:/ap=B0111$
# Disallow:/bbsId=0007$
# Disallow:/bbsId=0111$
# Disallow:/download.do
# Disallow:/dong.do
# Disallow:/file/
# Disallow:/post/
# Crawl-delay: 10
# Disallow:/eminwon.do